In [ ]:
import pandas as pd
import numpy as np
import spacy

import sys
import re, string
import time
import datetime

from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, roc_auc_score, accuracy_score, classification_report, f1_score

import torch
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
from torchtext import data
from transformers import AutoModel, AutoTokenizer
from transformers import BertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler


import matplotlib.pyplot as plt
import seaborn as sns

from collections import Counter

import warnings 
warnings.filterwarnings("ignore")

Notebook used for sentence classification with BERT. You will find below some links to understand how bert works :<br>
http://jalammar.github.io/illustrated-bert/<br>
http://jalammar.github.io/illustrated-transformer/ <br>
https://towardsdatascience.com/breaking-bert-down-430461f60efb

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

def format_time(elapsed):
    '''
    Takes a time in seconds and returns a string hh:mm:ss
    '''
    # Round to the nearest second.
    elapsed_rounded = int(round((elapsed)))
    
    # Format as hh:mm:ss
    return str(datetime.timedelta(seconds=elapsed_rounded))
def remove_URL(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return re.sub(url, '',text)
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)
def remove_nonalnum(x):
    pattern = re.compile('[^a-zA-Z]+')
    return pattern.sub(' ', x)

In [ ]:
def evaluate_clf(y_true, y_pred, experiment = "nlp_kaggle",run_name = "run1", save=False):
    c_true = Counter(y_true)
    c_pred = Counter(y_pred)
    for i in c_true.keys():
        c_true[i] = round(c_true[i]/len(y_true),2)
    for j in c_pred.keys():
        c_pred[j] = round(c_pred[j]/len(y_pred),2)   
    d_res = classification_report(y_true, y_pred, output_dict=True)
    if save==True:
        mlflow.set_experiment(f"/{experiment}")
        with mlflow.start_run(run_name= run_name):
            mlflow.log_param("value_count_true", c_true)
            mlflow.log_param("value_count_pred", c_pred)
            mlflow.log_param("population", len(y_true))
            for label in np.unique(y_true):
                mlflow.log_metric(f"prec_{label}", d_res[str(label)]["precision"])
                mlflow.log_metric(f"recall_{label}",  d_res[str(label)]["recall"])
            if len(np.unique(y_true)) == 2:
                mlflow.log_metric("auc", roc_auc_score(y_true, y_pred))
                mlflow.log_metric("f1_score_pos" , f1_score(y_true, y_pred))
    else:
        print("not saving")
    print(classification_report(y_true, y_pred))
def evaluate_clf_simple(y_true, y_pred):
    auc = roc_auc_score(y_true, y_pred)
    f1_score_pos  = f1_score(y_true, y_pred)
    print(f"the auc is {auc} and the f1 score is {f1_score_pos}")
    return auc

In [ ]:
from google.colab import files 
uploaded = files.upload()

Saving test.csv to test (1).csv
Saving train.csv to train (1).csv


The data used is from the kaggle competition <b>Real or Not? NLP with Disaster Tweets<b><br> https://www.kaggle.com/c/nlp-getting-started

In [ ]:
df = pd.read_csv("train.csv")
df_test = pd.read_csv("test.csv")

In [ ]:
df.head(1)

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1


In [ ]:
df["target"].value_counts(normalize=True, dropna=False)

0    0.57034
1    0.42966
Name: target, dtype: float64

In [ ]:
for col in ["text", "keyword"]:
    df[col] = df[col].apply(lambda x : remove_URL(x) if isinstance(x, str) else x)
    df[col] = df[col].apply(lambda x : remove_html(x) if isinstance(x, str) else x)
    df_test[col] = df_test[col].apply(lambda x : remove_URL(x) if isinstance(x, str) else x)
    df_test[col] = df_test[col].apply(lambda x : remove_html(x) if isinstance(x, str) else x)


df['keyword']=df['keyword'].apply(lambda x : remove_nonalnum(x) if isinstance(x, str) else x)
df_test['keyword']=df_test['keyword'].apply(lambda x : remove_nonalnum(x) if isinstance(x, str) else x)

In [ ]:
df["text"] = df["text"]+" "+df["keyword"].fillna("")

# Transformer model

In [ ]:
if torch.cuda.is_available():    
    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla K80


Many models are available like : bert-base-uncased/ bert-base-chinese/ gpt2/ roberta-base

In [ ]:
# Store the model we want to use
MODEL_NAME = "bert-base-uncased"

# We need to create the model and tokenizer
bert_model = AutoModel.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [ ]:
# or a more direct way 
tokens_pt2 = tokenizer.encode_plus("This is an input example. The one ", return_tensors="pt", add_special_tokens=True,
                                   max_length = 64,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True)

for key, value in tokens_pt2.items():
    print("{}:\n\t{}".format(key, value))

Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'only_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you may want to check this is the right behavior.


input_ids:
	tensor([[ 101, 2023, 2003, 2019, 7953, 2742, 1012, 1996, 2028,  102,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]])
token_type_ids:
	tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])
attention_mask:
	tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])


We are required to:
<br>
Add special tokens to the start and end of each sentence.<br>
Pad & truncate all sentences to a single constant length.<br>
Explicitly differentiate real tokens from padding tokens with the "attention mask".<br>

BERT is trained to only use this [CLS] token for classification, we know that the model has been motivated to encode everything it needs for the classification step into that single 768-value embedding vector. It's already done the pooling for us!

<strong>token_type_ids: This tensor will map every tokens to their corresponding segment.<br>
attention_mask: This tensor is used to "mask" padded values in a batch of sequence with different lengths.

In [ ]:
# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in df["text"].values:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 200,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

input_test = input_ids.copy()
# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(df["target"].values)

# Print sentence 0, now as a list of IDs.
print('Original: ', df["text"].values[0])
print('Token IDs:', input_ids[0])

# Building model structure


In [ ]:
class BertSentClassification(nn.Module):
    def __init__(self, bert_model, hidden_dim,output_dim, n_layers, bidirectional,dropout):
    super().__init__()
    self.bert_model = bert_model
    embedding_dim = bert_model.config.to_dict()['hidden_size']
    print(f"the embedding dimension of bert is {embedding_dim}")
    self.rnn = nn.GRU(embedding_dim,
                          hidden_dim,
                          num_layers = n_layers,
                          bidirectional = bidirectional,
                          batch_first = True,
                          dropout = 0 if n_layers < 2 else dropout)
    self.out = nn.Linear(hidden_dim * 2 if bidirectional else hidden_dim, output_dim)
    self.dropout = nn.Dropout(dropout)
    def forward(self, x, attention_masks):
        #x = [batch size, sent len]        
        with torch.no_grad():
            embedded = self.bert_model(x, attention_masks)[0]     
        #embedded = [batch size, sent len, emb dim]
        _, hidden = self.rnn(embedded)
        print(hidden.shape)
        #hidden = [n layers * n directions, batch size, emb dim]
        if self.rnn.bidirectional:
            hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        else:
            hidden = self.dropout(hidden[-1,:,:])       
        #hidden = [batch size, hid dim]
        output = self.out(hidden)
        #output = [batch size, out dim]
        return output

In [ ]:
OUTPUT_DIM = 1
HIDDEN_DIM = 256
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.1
epochs = 5
LEARNING_RATE = 1e-5
batch_size =  16 #. For fine-tuning BERT on a specific task, the authors recommend a batch 
# size of 16 or 32.

In [ ]:
# Combine the training inputs into a TensorDataset.
dataset = TensorDataset(input_ids, attention_masks, labels)

# Create a 90-10 train-validation split.

# Calculate the number of samples to include in each set.
train_size = int(0.9 * len(dataset))
val_size = len(dataset) - train_size

# Divide the dataset by randomly selecting samples.
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

print('{:>5,} training samples'.format(train_size))
print('{:>5,} validation samples'.format(val_size))
# The DataLoader needs to know our batch size for training, so we specify it 
# here

# Create the DataLoaders for our training and validation sets.
# We'll take training samples in random order. 
train_dataloader = DataLoader(
            train_dataset,  # The training samples.
            sampler = RandomSampler(train_dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

# For validation the order doesn't matter, so we'll just read them sequentially.
validation_dataloader = DataLoader(
            val_dataset, # The validation samples.
            sampler = SequentialSampler(val_dataset), # Pull out batches sequentially.
            batch_size = batch_size # Evaluate with this batch size.
        )
global_dataloader = DataLoader(
            dataset,  # The training samples.
            sampler = RandomSampler(dataset), # Select batches randomly
            batch_size = batch_size # Trains with this batch size.
        )

6,851 training samples
  762 validation samples


In [ ]:
model = BertSentClassification(bert_model,
                         HIDDEN_DIM,
                         OUTPUT_DIM,
                         N_LAYERS,
                         BIDIRECTIONAL,
                         DROPOUT)
model.cuda()

the embedding dimension of bert is 768


BertSentClassification(
  (bert_model): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementw

In [ ]:
nb_params0 = count_parameters(model)
for name, param in model.named_parameters():                
    if name.startswith('bert'):
        param.requires_grad = False
nb_params1 = count_parameters(model)
print((nb_params0, nb_params1))

(112241409, 2759169)


In [ ]:
def check_grad(model):
    for name, param in model.named_parameters():                
        if  param.requires_grad == True:
            print(f"gradients{param.grad}" )

In [ ]:
check_grad(model)

In [ ]:
optimizer = AdamW(model.parameters(),
                  lr =LEARNING_RATE, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                )
loss = nn.BCEWithLogitsLoss()
#Create a schedule with a learning rate that decreases linearly after linearly increasing during a warmup period.
# Total number of training steps is [number of batches] x [number of epochs]. 
# (Note that this is not the same as the number of training samples).
total_steps = len(train_dataloader) * epochs
# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 1000, # Default value in run_glue.py
                                            num_training_steps = total_steps)
loss = loss.to(device)

# Training the model

In [ ]:
import random
# This training code is based on the `run_glue.py` script here:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128

# Set the seed value all over the place to make this reproducible.
seed_val = 42

random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

# We'll store a number of quantities such as training and validation loss, 
# validation accuracy, and timings.
training_stats = []

# Measure the total training time for the whole run.
total_t0 = time.time()

# For each epoch...
for epoch_i in range(0, epochs):
    
    # ========================================
    #               Training
    # ========================================
    
    # Perform one full pass over the training set.

    print("")
    print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print('Training...')

    # Measure how long the training epoch takes.
    t0 = time.time()

    # Reset the total loss for this epoch.
    total_train_loss = 0

    # Put the model into training mode. Don't be mislead--the call to 
    # `train` just changes the *mode*, it doesn't *perform* the training.
    # `dropout` and `batchnorm` layers behave differently during training
    # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
    model.train()

    # For each batch of training data...
    for step, batch in enumerate(train_dataloader):

        # Progress update every 40 batches.
        if step % 40 == 0 and not step == 0:
            # Calculate elapsed time in minutes.
            elapsed = format_time(time.time() - t0)
            
            # Report progress.
            print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using the 
        # `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)

        check_grad(model)
        model.zero_grad()        
        logits = model(b_input_ids, b_input_mask)
        # Accumulate the training loss over all of the batches so that we can
        # calculate the average loss at the end. `loss` is a Tensor containing a
        # single value; the `.item()` function just returns the Python value 
        # from the tensor.
        
        logits_loss = logits.view(-1).to(device)

        loss_val = loss(logits_loss, b_labels.float())
        total_train_loss += loss_val.item()

        # Perform a backward pass to calculate the gradients.
        loss_val.backward()

        # Clip the norm of the gradients to 1.0.
        # This is to help prevent the "exploding gradients" problem.
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # Update parameters and take a step using the computed gradient.
        # The optimizer dictates the "update rule"--how the parameters are
        # modified based on their gradients, the learning rate, etc.
        optimizer.step()

        # Update the learning rate.
        scheduler.step()

    # Calculate the average loss over all of the batches.
    avg_train_loss = total_train_loss / len(train_dataloader)            
    
    # Measure how long this epoch took.
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))
        
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.

    print("")
    print("Running Validation...")

    t0 = time.time()

    # Put the model in evaluation mode--the dropout layers behave differently
    # during evaluation.
    model.eval()

    # Tracking variables 
    total_eval_auc = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    # Evaluate data for one epoch
    for batch in validation_dataloader:
        
        # Unpack this training batch from our dataloader. 
        #
        # As we unpack the batch, we'll also copy each tensor to the GPU using 
        # the `to` method.
        #
        # `batch` contains three pytorch tensors:
        #   [0]: input ids 
        #   [1]: attention masks
        #   [2]: labels 
        b_input_ids = batch[0].to(device)
        b_input_mask = batch[1].to(device)
        b_labels = batch[2].to(device)
        
        # Tell pytorch not to bother with constructing the compute graph during
        # the forward pass, since this is only needed for backprop (training).
        with torch.no_grad():        

            # Forward pass, calculate logit predictions.
            # token_type_ids is the same as the "segment ids", which 
            # differentiates sentence 1 and 2 in 2-sentence tasks.
            # The documentation for this `model` function is here: 
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            # Get the "logits" output by the model. The "logits" are the output
            # values prior to applying an activation function like the softmax.
            logits = model(b_input_ids, b_input_mask)
            
        # Accumulate the validation loss.
        loss_res = loss(logits.view(-1), b_labels.float())
        total_eval_loss += loss_res.item()

        # Move logits and labels to CPU
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
    

        # Calculate the aucfor this batch of test sentences, and
        # accumulate it over all batches.
        total_eval_auc += evaluate_clf_simple(label_ids, np.concatenate([ np.where(x >0.5, 1,0) for x in logits]))
        

    avg_val_auc = total_eval_auc / len(validation_dataloader)
    print("  Auc: {0:.2f}".format(avg_val_auc))

    # Calculate the average loss over all of the batches.
    avg_val_loss = total_eval_loss / len(validation_dataloader)
    
    # Measure how long the validation run took.
    validation_time = format_time(time.time() - t0)
    
    print("  Validation Loss: {0:.2f}".format(avg_val_loss))
    print("  Validation took: {:}".format(validation_time))

    # Record all statistics from this epoch.
    training_stats.append(
        {
            'epoch': epoch_i + 1,
            'Training Loss': avg_train_loss,
            'Valid. Loss': avg_val_loss,
            'Valid. Auc.': avg_val_auc,
            'Training Time': training_time,
            'Validation Time': validation_time
        }
    )

print("")
print("Training complete!")

print("Total training took {:} (h:mm:ss)".format(format_time(time.time()-total_t0)))

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
        -3.8998e-05, -9.3672e-05,  6.6222e-05, -2.5758e-05, -1.5974e-04,
         7.7898e-06,  1.2152e-05,  9.6659e-05,  3.8600e-05,  1.5934e-04,
         7.5075e-06,  1.3347e-05, -5.8738e-06, -1.1931e-04,  1.2073e-04,
         1.7331e-04,  2.4585e-04,  1.6187e-04,  7.7397e-05,  9.0026e-05,
        -5.1907e-05, -1.6949e-05,  5.4551e-05, -1.1595e-04,  5.6654e-05,
        -2.7495e-05,  3.8748e-05,  4.8067e-05,  1.0980e-05,  2.4457e-05,
        -2.7760e-05,  1.2116e-05,  1.3121e-04, -1.7433e-07,  7.6727e-05,
        -1.1182e-05,  1.6813e-05, -1.1867e-04,  9.6361e-06, -3.5680e-06,
        -1.2181e-04, -5.7511e-05, -6.6890e-05,  9.7561e-05,  1.6922e-04,
         5.7547e-05, -1.7586e-04, -7.0905e-06,  2.0210e-04,  1.7227e-04,
         4.3636e-05, -5.8181e-05,  1.7731e-05,  1.0902e-05, -6.2725e-05,
         4.5368e-06,  6.4647e-05,  4.3455e-05, -3.0175e-04,  1.3129e-04,
        -1.8420e-05,  1.0797e-04,  2.9584e-05,

KeyboardInterrupt: ignored

In [ ]:
# Create sentence and label lists
sentences = df_test["text"].values
df_test["target"] = 0
labels = df_test["target"].values

# Tokenize all of the sentences and map the tokens to thier word IDs.
input_ids = []
attention_masks = []

# For every sentence...
for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = 200,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
    # Add the encoded sentence to the list.    
    input_ids.append(encoded_dict['input_ids'])
    
    # And its attention mask (simply differentiates padding from non-padding).
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# Set the batch size.  
batch_size = 256

# Create the DataLoader.
prediction_data = TensorDataset(input_ids, attention_masks, labels)
prediction_sampler = SequentialSampler(prediction_data)
prediction_dataloader = DataLoader(prediction_data, sampler=prediction_sampler, batch_size=batch_size)
# Prediction on test set

print('Predicting labels for {:,} test sentences...'.format(len(input_ids)))

# Put model in evaluation mode
model.eval()

# Tracking variables 
predictions , true_labels = [], []

# Predict 
for batch in prediction_dataloader:
  # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
  # Telling the model not to compute or store gradients, saving memory and 
  # speeding up prediction
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      outputs = model(b_input_ids, b_input_mask)

    logits = outputs

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    # Store predictions and true labels
    predictions.append(logits)
    true_labels.append(label_ids)

print('    DONE.')

Predicting labels for 3,263 test sentences...
    DONE.


In [ ]:
pred_test = np.concatenate([ w.flatten() for w in predictions])
df_test["pred"] = pred_test
df_test["pred_proba"] = df_test["pred"].apply(lambda x : 1/(1+np.exp(-x)))
#submit with text col only
df_test2 = df_test.copy(deep=True)
df_test2["target"] = df_test2["pred_proba"].apply(lambda x : 1 if x >0.5 else 0 )

In [ ]:
path = "gdrive/My Drive/Colab Notebooks/"
df_test2.loc[:, ["id", "target"]].to_csv(path+"submission_text_only_8.csv", index=False, sep=",")